In [128]:
# from thermo.chemical import Chemical

# emissivity = 0
# temperature = 1900 #kelvin

# boltzmann_constant = 5.6703e-10 

# test this is 

# aluminium = Chemical('aluminium')

# aluminium_vapor_pressure = aluminium.VaporPressure.calculate(2000,"BOILING_CRITICAL") # takes temp in K, returns pressure in Pa
#                                                                                       # not accurate!
# print(aluminium_vapor_pressure)

# graphite = Chemical('carbon|solid')
# graphite_vapor_pressure = graphite.VaporPressure.T_dependent_property(1900) # takes temp in K, returns pressure in Pa
# print(graphite_vapor_pressure)

In [129]:
#Boltzman power consumption


emissivity = 0
temperature = 1900 #kelvin
boltzmann_constant = 5.6703e-10


In [130]:
from mendeleev import element
import sympy.physics.units as units
import scipy.constants as constants
amu = constants.physical_constants["atomic mass unit-kilogram relationship"][0]
import math
import sys
sys.path.insert(0, '../../files/ionprinter/simulation/utilities/')
import util

In [131]:
#Bowtie inputs:

print_species = element('Al')

print_speed = 10.0 #g/hour

print_speed /= 1000.0
print_speed /= 3600.0

number_of_bowties = 128

bowtie_nozzle_radius = 0.00033#m
bowtie_hot_area = 0.001 #m

bowtie_temperature = 2000.0 #Kelvin

# import pandas as pd
# data = [[1, 2], [3, 4]]
# pd.DataFrame(data, columns=["Foo", "Bar"])


#chamber inputs

ionization_chamber_radius = 0.0015 #m
ionization_chamber_length = 0.01 #m
acceleration_voltage = 15 #v

build_platform_distance = 0.03 #m

In [132]:
atoms_per_second = print_speed/(print_species.mass*amu)
moles_per_second = atoms_per_second/constants.Avogadro
print_volume = ((print_speed*3600.0)/(print_species.density*1000.0))/1.0e-6

total_beam_current = atoms_per_second*constants.elementary_charge
per_chamber_beam_current = total_beam_current/number_of_bowties

print("Atoms per second: %0.5g atoms/s" % atoms_per_second)
print("Moles per second: %0.5g M/s" % moles_per_second)
print("Print volume: %0.5g cm^3/s" % print_volume)
print("Total beam current: %0.5g Amps" % total_beam_current)
print("Per-chamber current: %0.5g Amps" % per_chamber_beam_current)

#I find it infinitely amusing that "40 amps of aluminum" is "40 grams per second of aluminum" within 0.6%

Atoms per second: 6.1999e+19 atoms/s
Moles per second: 0.00010295 M/s
Print volume: 3.7052 cm^3/s
Total beam current: 9.9333 Amps
Per-chamber current: 0.077604 Amps


Determine RMS neutral gas velocity at a certain temperature:

$$\sqrt{\frac{3RT}{m}}$$

In [133]:
bowtie_rms_neutral_velocity = math.sqrt((3*constants.Boltzmann*bowtie_temperature)/(print_species.mass*amu))
print("RMS particle velocity: %0.5g m/s" % bowtie_rms_neutral_velocity)

RMS particle velocity: 1359.8 m/s


Determine approximate neutral gas pressure:

(I'm not even sure that you can do this.)
$$\text{PV}=\text{nRT}$$

In [134]:
R_constant = constants.Boltzmann*constants.Avogadro

nozzle_volume = (math.pi*(bowtie_nozzle_radius**2))*bowtie_rms_neutral_velocity

bowtie_nozzle_pressure = ((moles_per_second/number_of_bowties)*R_constant*bowtie_temperature)/(nozzle_volume)

mean_free_path = 1.0/(((atoms_per_second/number_of_bowties)/nozzle_volume)*math.pi*((print_species.vdw_radius*1e-12)**2))

print("Bowtie nozzle pressure: %0.5g Pa" % bowtie_nozzle_pressure)
print("Approximate nozzle mean free path: %0.5g m" % mean_free_path)


print("Knudsen number: local %0.5g, global %0.5g" % ((mean_free_path/bowtie_nozzle_radius),
                                                     (mean_free_path/build_platform_distance)))



Bowtie nozzle pressure: 28.751 Pa
Approximate nozzle mean free path: 0.0090298 m
Knudsen number: local 27.363, global 0.30099


In [135]:
acceleration_velocity = math.sqrt((2.0*15*constants.electron_volt)/(print_species.mass*amu))
print("Accel. vel: %0.5g m/s" % acceleration_velocity)
print("Accel. power per chamber: %0.5g W, total %0.5g W " % (acceleration_voltage*per_chamber_beam_current
                                                        ,acceleration_voltage*total_beam_current))

beam_exit_velocity = bowtie_rms_neutral_velocity+acceleration_velocity

focus_field = util.scharge_efield(per_chamber_beam_current,beam_exit_velocity,ionization_chamber_radius)
print("Required focus field: %0.5g V/m" % focus_field)

#sine focus voltage equation would be nice
print("Required focus voltage: ~%0.5g V" % (focus_field*(ionization_chamber_radius/5.0)))


Accel. vel: 10358 m/s
Accel. power per chamber: 1.1641 W, total 149 W 
Required focus field: 7.9403e+07 V/m
Required focus voltage: ~23821 V
